# Install Packages and Download Dataset

**https://universe.roboflow.com/carvajal/cartas-poker/browse?queryText=&pageSize=200&startingIndex=400&browseQuery=true**

In [1]:
# Roboflow for Dataset
!pip install roboflow

# YOLOV8
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-ma

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="ueSrp8zT1NmWHzCxoYmt")
project = rf.workspace("carvajal").project("cartas-poker")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.75, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Cartas-Poker-2 in yolov8:: 100%|██████████| 6434/6434 [00:01<00:00, 6337.42it/s]


In [3]:
import ultralytics

# Check GPU
ultralytics.checks()

Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 33.8/78.2 GB disk)


# Fix YAML Paths

In [4]:
import yaml
import os

def edit_yaml_paths(yaml_file_path):
    # Dataset main directory
    root_dir = os.path.dirname(yaml_file_path)

    # Load the YAML file
    with open(yaml_file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Modify YAML file to include correct subset paths
    data['train'] = os.path.join(root_dir, 'train/images')
    data['val'] = os.path.join(root_dir, 'valid/images')
    data['test'] = os.path.join(root_dir, 'test/images')

    with open(yaml_file_path, 'w') as file:
        yaml.safe_dump(data, file)

    print("YAML file updated successfully!")


In [5]:
yaml_path = '/content/Cartas-Poker-2/data.yaml'
edit_yaml_paths(yaml_path)

YAML file updated successfully!


# Create, Train, and Validate Model

In [6]:
from ultralytics import YOLO
#weights_path = 'yolov8n.pt'  # pretrained model (recommended for training)
#weights_path = 'runs/detect/train5/weights/best.pt' # load weights from previous training
weights_path = '/content/best.pt'

# Load a model
model = YOLO(weights_path)
print("Model successfully loaded")

Model successfully loaded


In [7]:
# TRAIN, VALIDATE, AND TEST MODEL
training = model.train(data=yaml_path, epochs=15)
validation = model.val()
testing = model.val(data=yaml_path, split='test')

Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/best.pt, data=/content/Cartas-Poker-2/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 116MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 6.25M/6.25M [00:00<00:00, 269MB/s]


AMP: checks passed ✅


...

                   all        193        838      0.986      0.986      0.994      0.912
           10 Diamonds         16         16          1      0.964      0.995       0.97
             10 Hearts         17         17      0.988          1      0.995      0.895
             10 Spades         15         15      0.996          1      0.995      0.902
           10 Trefoils         18         18      0.997          1      0.995      0.905
            2 Diamonds         16         16      0.933          1      0.995      0.935
              2 Hearts         17         17          1      0.967      0.995      0.876
              2 Spades         15         15      0.927          1      0.995      0.868
            2 Trefoils         18         18          1      0.839      0.987      0.883
            3 Diamonds         16         16          1      0.956      0.995      0.925
              3 Hearts         17         17      0.998          1      0.995      0.899
              3 Spade

val: Scanning /content/Cartas-Poker-2/test/labels... 128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 1322.51it/s]

val: New cache created: /content/Cartas-Poker-2/test/labels.cache



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.68it/s]


                   all        128        555      0.976      0.979      0.987      0.912
           10 Diamonds         10         10      0.989          1      0.995      0.965
             10 Hearts         11         11      0.983          1      0.995      0.876
             10 Spades         10         10          1      0.924      0.995      0.919
           10 Trefoils         12         12      0.986          1      0.995      0.942
            2 Diamonds         10         10          1      0.948      0.995      0.901
              2 Hearts         11         11      0.902      0.909      0.877      0.786
              2 Spades         10         10          1      0.989      0.995       0.82
            2 Trefoils         12         12      0.964      0.917      0.979      0.922
            3 Diamonds         10         10      0.979          1      0.995      0.881
              3 Hearts         11         11      0.906      0.883       0.94      0.844
              3 Spade

# Zip weights folder to be downloaded

In [8]:
import shutil
from IPython.display import FileLink


directory_to_zip = "/content/runs"
output_zip = "/content/runs.zip"

# Zip the directory
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', directory_to_zip)

# Check if the zip file was created successfully
if os.path.exists(output_zip):
    print(f"Successfully created zip file: {output_zip}")
    # Provide a link to download the zip file
    display(FileLink(output_zip))
else:
    print("Failed to create zip file")


Successfully created zip file: /content/runs.zip


/content/runs.zip